In [11]:
from rotating_coil_analyzer.gui.app import build_catalog_gui
gui = build_catalog_gui()
gui


In [ ]:
import rotating_coil_analyzer.ingest.readers_sm18 as m
print(m.__file__)
print(m.Sm18CorrSigsReader)
